<img src="figures/hiti.png" alt="HITILab" width="150"/>

<!-- Author: Theo Dapamede, MD, PhD -->
<!-- Github: theodapamede -->

# EMBED: Data Preprocessing

By going through this lecture and notebook, you should be able to:

1. Identify specific DICOM tags for mammography;
2. Understand the relationship between the DICOM tags and the EMBED `metadata.csv`;
3. Perform common image preprocessing techniques for mammograms;
4. Identify common pitfalls or challenges in preprocessing mammograms and understand potential solutions; 
5. Understand the basics of DBT images;
6. Understand the basics of MRI images

# 0. Load Libraries and Prepare Environment

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.animation import FuncAnimation
from PIL import Image
from IPython.display import display

In [ ]:
from tqdm import tqdm

In [ ]:
import pydicom
from pydicom.pixel_data_handlers import apply_modality_lut, apply_voi_lut

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [ ]:
rc('animation', html='jshtml')

In [ ]:
RANDOM_SEED = 2024

## 0.1. Defining Functions

In [ ]:
def process_dicom_image(dicom):
    return apply_voi_lut(apply_modality_lut(dicom.pixel_array, dicom), dicom)

In [ ]:
def animate_3d_array(data, DPI=150, vmin=np.iinfo(np.int16).min, vmax=np.iinfo(np.int16).max, aspect_ratio = "equal"):
    # Ensure data is a 3D numpy array
    if not isinstance(data, np.ndarray) or data.ndim != 3:
        raise ValueError("Input must be a 3D numpy array")

    fig, ax = plt.subplots(dpi=DPI)
    plt.close(fig)
    
    # Display the first slice
    im = ax.imshow(data[0], cmap='gray', animated=True, vmin=vmin, vmax=vmax, aspect=aspect_ratio)
    
    # Set the title to show the current slice number
    title = ax.set_title('Slice 0')

    def update(frame):
        im.set_array(data[frame])
        title.set_text(f'Slice {frame}')
        return im, title

    # Create the animation
    anim = FuncAnimation(fig, update, frames=data.shape[0], interval=50, blit=True)
    
    return anim

# 1. Load Tables

In [ ]:
metadata = pd.read_csv('/fsx/embed/summer-school-24/Theo_session/tables/tutorial_metadata.csv')

In [ ]:
# Sample a dicom file for example purposes
dicom_file = metadata.anon_dicom_path.iloc[3]
dicom_file

In [ ]:
loaded_dicom = pydicom.dcmread(dicom_file)

# 1.1. Mammogram specific tags

- PatientID
- AccessionNumber
- StudyDate
- UIDs [Series, Study, Instance]
- Modality
- Study and Series Description
- Breast Implant Present
- Compression Force and Compression Pressure
- Image Laterality
- View Position

In [ ]:
# DICOM Tags --> in parentheses (xxxx, xxxx)
# Followed by DICOM Tag name; Value Representation; Value
loaded_dicom

### 1.1.1. Modality

In [ ]:
loaded_dicom.Modality

### 1.1.2. Breast Implant Present

In [ ]:
loaded_dicom.BreastImplantPresent

### ☢️ ***Exercise 1***

*An ongoing challenge is that this tag does not always tell the truth.*

What solution(s) could address this issue?

In [ ]:
# Type answer here

### 1.1.3 Compression force & Compression Pressure
Compression force typically ranges from 100-200 Newtons. 

Importance of adequate compression force:
1. Reducing radiation dose
2. Improve image quality

Compression pressure (in kPa) is related to pain during image acquitision. This tag is reported to be available in more recent machines.

Some questions to think about:
- How does compression force or compression pressure affect the screening adherance rate?
- Does this rate vary between different demographics?
- Is it related to density of the breast?

*Reference*

- [Moshina, et al. 2018](https://www.sciencedirect.com/science/article/pii/S0091743518300082)

In [ ]:
loaded_dicom.CompressionForce

In [ ]:
# loaded_dicom.CompressionPressure  # not all vendors provide this value

### 1.1.5. Image Laterality

In [ ]:
loaded_dicom.ImageLaterality

### 1.1.6. View Position

In [ ]:
loaded_dicom.ViewPosition

There is another tag that can also provide information regarding view position.

In [ ]:
loaded_dicom.ViewCodeSequence

In [ ]:
loaded_dicom.ViewCodeSequence[0]

# 1.2. Nested DICOM Tags

![Nested DICOM Tags](https://i.sstatic.net/JMyoM.gif, "Nested DICOM Tags")

Image Source: https://stackoverflow.com/questions/46690392/how-to-read-nested-child-dicom-tags-from-sequences-using-fo-dicom

In [ ]:
loaded_dicom.ViewCodeSequence

In [ ]:
for sequence in loaded_dicom.ViewCodeSequence:
    print(sequence)

In [ ]:
loaded_dicom.ViewCodeSequence[0].CodeMeaning

[Click to see more information regarding Coding Schemes](https://dicom.nema.org/medical/dicom/current/output/chtml/part16/chapter_8.html#chapter_8.1)

### ☢️ ***Exercise 2***

If `loaded_dicom.ViewPosition` contains the same information as `loaded_dicom.ViewCodeSequence[0].CodeMeaning`, which tag do you use? Why?

In [ ]:
# Type answer here

**Another example**

In [ ]:
loaded_dicom.SourceImageSequence

In [ ]:
loaded_dicom.SourceImageSequence[0]

In [ ]:
loaded_dicom.SourceImageSequence[0].PurposeOfReferenceCodeSequence[0]

In [ ]:
loaded_dicom.SourceImageSequence[0].PurposeOfReferenceCodeSequence[0].CodeMeaning

# 2.1. The Structure of EMBED `Metadata.csv`

The majority of columns in `metadata.csv` are extracted DICOM tags.

In [ ]:
for i in range(len(metadata.columns)):
    print(i+1, list(metadata.columns)[i])

In `metadata` file, the nested DICOM tags follow the following structure:

`0_[Parent Tag]_[Child Tag]`.

For example: '0_AnatomicRegionSequence_CodingSchemeDesignator'

# 3. Working with DICOM Images

## 3.1. Load DICOM Image Pixels

In [ ]:
loaded_dicom[0x7FE0,0x0010]

In [ ]:
loaded_dicom[0x7FE0,0x0010].keyword

In [ ]:
loaded_dicom.pixel_array

In [ ]:
image = loaded_dicom.pixel_array

In [ ]:
print(image.shape)  # (Height, Width)

In [ ]:
print(image.dtype)

## 3.2 View a DICOM Image

In [ ]:
plt.figure(dpi=300)
plt.imshow(loaded_dicom.pixel_array, 'gray')
plt.show()

# 4. Common Image Preprocessing Techniques in Mammography for Deep Learning

# 4.1. Applying Modality LUT and VOI LUT

Review of the Modality LUT and VOI LUT discussed on Monday

### 4.1.1. Applying Modality Transforms
The raw Pixel Data in a DICOM file may not be in the modality units. Therefore, we first need to apply a modality transformation to standardize the units.

There are 2 ways of transforming the values:
1. Using the Rescale Intercept and Rescale Slope
We apply the Rescale Slope and Intercept using the following equation:

$$\text{Output Value} = m \cdot \text{Stored Value} + b$$

where *m* is the Rescale Slope and *b* is the Rescale Intercept

2. Using the Modality LUT
This method uses a Look Up Table which will specify what a pixel value will be transformed into.

In a DICOM file, only one of the above method is available.


### 4.1.2. Applying the VOI LUT

![VOI LUT](figures/voi_lut.png)

In [ ]:
metadata.Manufacturer.value_counts()

In [ ]:
ge = metadata[metadata.Manufacturer.str.contains('GE')]
holo = metadata[metadata.Manufacturer.str.contains('HOLO')]

In [ ]:
ge.shape

In [ ]:
# sample_df = ge.sample(frac=1, random_state=RANDOM_SEED)
sample_df = ge.sample(3, random_state=RANDOM_SEED)
for i in range(sample_df.shape[0]):
    dcm = pydicom.dcmread(sample_df.anon_dicom_path.iloc[i])
    fig, axs = plt.subplots(1, 2, dpi=150, constrained_layout=True, sharey=True)
    axs[0].imshow(dcm.pixel_array, 'gray')
    axs[1].imshow(process_dicom_image(dcm), 'gray')
    axs[0].set_title("Raw")
    axs[1].set_title("Processed")
    plt.show()

In [ ]:
# sample_df = holo.sample(frac=1, random_state=RANDOM_SEED)
sample_df = holo.sample(3, random_state=RANDOM_SEED)
for i in range(sample_df.shape[0]):
    dcm = pydicom.dcmread(sample_df.anon_dicom_path.iloc[i])
    fig, axs = plt.subplots(1, 2, dpi=150, constrained_layout=True, sharey=True)
    axs[0].imshow(dcm.pixel_array, 'gray')
    axs[1].imshow(process_dicom_image(dcm), 'gray')
    axs[0].set_title("Raw")
    axs[1].set_title("Processed")
    plt.show()

### ☢️ ***Exercise 3***

Discuss with the person next to you, what you would do if you obtained an external dataset that has the VOI LUT removed?

# 4.2. Flipping images to face the same orientation

There are several reasons why we want to flip the images to face the same direction:

1. **Baseline**: Flipping the images to face the same direction creates a consistent baseline for further downstream tasks.
2. **Data augmentation**: In deep learning, we want to increase the variablity of the training data by performing random flippings, rotations, or scalings. With a consistent baseline, it is easier to control the augmentation pipeline.
3. **Comparative Analysis**: Aligning images in the same orientation makes it easier to compare different images. It is also an initial step to perform image registration.

The following cell is reused from: https://github.com/Emory-HITI/EMBED_Open_Data/blob/main/DCM_to_PNG.ipynb


In [ ]:
# Get DICOM image metadata
class DCM_Tags():
    def __init__(self, img_dcm):
        try:
            self.laterality = img_dcm.ImageLaterality
        except AttributeError:
            self.laterality = np.nan
            
        try:
            self.view = img_dcm.ViewPosition
        except AttributeError:
            self.view = np.nan
            
        try:
            self.orientation = img_dcm.PatientOrientation
        except AttributeError:
            self.orientation = np.nan

# Check whether DICOM should be flipped
def check_dcm(imgdcm):
    # Get DICOM metadata
    tags = DCM_Tags(imgdcm)
    
    # If image orientation tag is defined
    if ~pd.isnull(tags.orientation):
        # CC view
        if tags.view == 'CC':
            if tags.orientation[0] == 'P':
                flipHorz = True
            else:
                flipHorz = False
            
            if (tags.laterality == 'L') & (tags.orientation[1] == 'L'):
                flipVert = True
            elif (tags.laterality == 'R') & (tags.orientation[1] == 'R'):
                flipVert = True
            else:
                flipVert = False
        
        # MLO or ML views
        elif (tags.view == 'MLO') | (tags.view == 'ML'):
            if tags.orientation[0] == 'P':
                flipHorz = True
            else:
                flipHorz = False
            
            if (tags.laterality == 'L') & ((tags.orientation[1] == 'H') | (tags.orientation[1] == 'HL')):
                flipVert = True
            elif (tags.laterality == 'R') & ((tags.orientation[1] == 'H') | (tags.orientation[1] == 'HR')):
                flipVert = True
            else:
                flipVert = False
        
        # Unrecognized view
        else:
            flipHorz = False
            flipVert = False
            
    # If image orientation tag is undefined
    else:
        # Flip RCC, RML, and RMLO images
        if (tags.laterality == 'R') & ((tags.view == 'CC') | (tags.view == 'ML') | (tags.view == 'MLO')):
            flipHorz = True
            flipVert = False
        else:
            flipHorz = False
            flipVert = False
            
    return flipHorz, flipVert

In [ ]:
# Check if a horizontal flip is necessary
original_image = process_dicom_image(loaded_dicom)
horz, _ = check_dcm(loaded_dicom)
if horz:
    # Flip img horizontally
    flipped_image = np.fliplr(original_image)

In [ ]:
fig, axs = plt.subplots(1, 2, dpi=150, constrained_layout=True, sharey=True)
axs[0].imshow(original_image, 'gray')
axs[1].imshow(flipped_image, 'gray')
axs[0].set_title("Original")
axs[1].set_title("Flipped")
plt.show()

# 4.3. Masking out breast tissue

Some manufacturers burn in additional information to the images, such as ViewPosition markers (e.g. RMLO), dates, initials, etc.

We want to remove these markers to only obtain the breast tissue.

Why do we want to do this?

In [ ]:
def select_island(img_):
    _, bin_mask = cv2.threshold(img_, 0.05, 1, cv2.THRESH_BINARY)
    
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(bin_mask.astype(np.uint8), connectivity=8)
    
    # Filter out small components (you can adjust the threshold as needed)
    min_area_threshold = 20000  # minimum area to keep
    island_mask = np.zeros_like(bin_mask.astype(np.uint8))
    for label in range(1, num_labels):  # Exclude background (label 0)
        area = stats[label, cv2.CC_STAT_AREA]
        if area >= min_area_threshold:
            island_mask[labels == label] = 1

    return bin_mask, island_mask

In [ ]:
_, binary_mask = cv2.threshold(flipped_image, np.ptp(flipped_image) * 0.0005 + flipped_image.min(), flipped_image.max(), cv2.THRESH_BINARY)

In [ ]:
plt.figure(dpi=150)
plt.imshow(binary_mask, 'gray')
plt.show()

In [ ]:
num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_mask.astype(np.uint8), connectivity=8)

In [ ]:
# Get the area
largest_area = 0
label_with_largest_area = None
for i in range(1, num_labels):
    area = stats[i, cv2.CC_STAT_AREA]
    print("Label:", i, "--> Area:", area)
    if area > largest_area:
        largest_area = area
        label_with_largest_area = i

In [ ]:
island_mask = np.zeros_like(binary_mask.astype(np.uint8))
island_mask[labels == label_with_largest_area] = 1

In [ ]:
plt.figure(dpi=150)
plt.imshow(island_mask, 'gray')
plt.show()

In [ ]:
flipped_image_no_label = flipped_image * island_mask

fig, axs = plt.subplots(1, 4, dpi=200, constrained_layout=True, sharey=True)
axs[0].imshow(original_image, 'gray')
axs[1].imshow(flipped_image, 'gray')
axs[2].imshow(island_mask, 'gray')
axs[3].imshow(flipped_image_no_label, 'gray')
axs[0].set_title("1.\nOriginal")
axs[1].set_title("2.\nFlipped")
axs[2].set_title("3.\nBinary Mask")
axs[3].set_title("4.\nLabel Removed")
plt.show()

# 4.4. Crop to Breast Tissue

Several points of importance of cropping to breast tissue:

1. Surrounding air do not contain important information* --> saves disk space;
2. Provides ease and more flexibility if one requires further processing steps, for example padding;
2. Elimination of artefacts;

Here's an example of artifacts in air pixels which may be due to scattered radiation.

It is noticable in the Heatmap subimage that the model was picking up these artefacts as signal instead of the breast.

![](figures/air_artifacts.png "Example of the presence of scattered radiation ")


In [ ]:
def crop_zeros(image, img_mask_):
    points = np.argwhere(img_mask_.sum(axis=0))
    left_boundary = points.min()
    right_boundary = points.max()
    
    points = np.argwhere(img_mask_.sum(axis=1))
    top_boundary = points.min()
    bottom_boundary = points.max()

    return image[top_boundary:bottom_boundary, left_boundary: right_boundary]

In [ ]:
cropped_img = crop_zeros(flipped_image_no_label, island_mask)

In [ ]:
flipped_image_no_label = flipped_image * island_mask

fig, axs = plt.subplots(1, 4, dpi=200, constrained_layout=True, sharey=True)
axs[0].imshow(original_image, 'gray')
axs[1].imshow(flipped_image, 'gray')
axs[2].imshow(flipped_image_no_label, 'gray')
axs[3].imshow(cropped_img, 'gray')
axs[0].set_title("1.\nOriginal")
axs[1].set_title("2.\nFlipped")
axs[2].set_title("3.\nLabel Removed")
axs[3].set_title("4.\nFinal Image")
plt.show()

# 4.5. Saving to PNG

In [ ]:
# Convert pixel array to PNG as a 16-bit greyscale
image_to_save = cropped_img.astype(np.double)

# Rescale grey scale between 0-65535
image_to_save = (np.maximum(image_to_save, 0) / image_to_save.max()) * 65535.0

# Convert to uint16
image_to_save = np.uint16(image_to_save)

output_png_path = f"./output/{loaded_dicom.SOPInstanceUID}.png"

if not os.path.exists("./output/"):
    os.makedirs("./output/")

image = Image.fromarray(image_to_save.astype(np.uint16))
image.save(output_png_path)

# 4.6. Common Normalization Techniques

1. Min-max normalization
2. Standardization

# 4.6.1. Min-Max Normalization

The most common Min-Max Normalization technique is transforming the pixel distribution between 0 and 1.

The steps are as follows:

1. Calculate the minimum and maximum pixel values of the image
2. Subtract the image with it's minimum value
3. Divide the results with the range of pixel values, i.e. Maximum minus Minimum (max - min)

In [ ]:
img_max = cropped_img.max()
img_min = cropped_img.min()

In [ ]:
normalized_img = (cropped_img - img_min) / (img_max - img_min)

In [ ]:
normalized_img.min()

In [ ]:
normalized_img.max()

In [ ]:
print(f"Cropped Image: (min={cropped_img.min():.2f}, max={cropped_img.max():.2f})")
print(f"Normalized Image: (min={normalized_img.min():.2f}, max={normalized_img.max():.2f})")

fig, axs = plt.subplots(1, 2, dpi=300, constrained_layout=True, sharey=True)
axs[0].imshow(cropped_img, 'gray')
axs[1].imshow(normalized_img, 'gray')
axs[0].set_title("Cropped Image")
axs[1].set_title("Normalized Image")
plt.show()

# 4.6.2. Standardization

The most common standardization technique is transforming the pixel distribution to a mean of zero and a standard deviation of 1 (or unit variance).

The steps are as follows:

1. Calculate the mean and standard deviation of the image
2. Subtract the image with it's mean
3. Divide the results with the standard deviation

In [ ]:
img_mean = cropped_img.mean()
img_std = cropped_img.std()

In [ ]:
standard_img = (cropped_img - img_mean) / img_std

In [ ]:
standard_img.mean()

In [ ]:
standard_img.std()

In [ ]:
print(f"Cropped Image: (mean={cropped_img.mean():.2f}, std={cropped_img.std():.2f})")
print(f"Standardized Image: (mean={standard_img.mean():.2f}, std={standard_img.std():.2f})")

fig, axs = plt.subplots(1, 2, dpi=300, constrained_layout=True, sharey=True)
axs[0].imshow(cropped_img, 'gray')
axs[1].imshow(standard_img, 'gray')
axs[0].set_title("Cropped Image")
axs[1].set_title("Standardized Image")
plt.show()

# 5. DIGITAL BREAST TOMOSYNTHESIS (DBT)

In [ ]:
dbt_files = glob.glob("/fsx/embed/summer-school-24/Theo_session/dicoms/dbt/*.dcm")

In [ ]:
dbt_dcm = pydicom.dcmread(dbt_files[0])

In [ ]:
dbt_dcm

In [ ]:
dbt_image_stack = apply_voi_lut(apply_modality_lut(dbt_dcm.pixel_array, dbt_dcm), dbt_dcm)

In [ ]:
fig, axs = plt.subplots(6, 7, figsize=(20, 20), sharey=True, sharex=True, tight_layout=True)

for i, ax in enumerate(axs.flatten()):
    ax.imshow(np.rot90(dbt_image_stack[i], 2), 'gray')
    ax.set_axis_off()
plt.show()

In [ ]:
animate_3d_array(np.rot90(dbt_image_stack, 2), vmin=dbt_image_stack.min(), vmax=dbt_image_stack.max())

## 5.2 Crop DBT

In [ ]:
dbt_image_stack_sum = dbt_image_stack.sum(0)

In [ ]:
_, binary_mask = cv2.threshold(dbt_image_stack_sum, np.ptp(dbt_image_stack_sum) * 0.0005 + dbt_image_stack_sum.min(), dbt_image_stack_sum.max(), cv2.THRESH_BINARY)

In [ ]:
# Get the area
num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_mask.astype(np.uint8), connectivity=8)

largest_area = 0
label_with_largest_area = None
for i in range(1, num_labels):
    area = stats[i, cv2.CC_STAT_AREA]
    print("Label:", i, "--> Area:", area)
    if area > largest_area:
        largest_area = area
        label_with_largest_area = i

In [ ]:
island_mask = np.zeros_like(binary_mask.astype(np.uint8))
island_mask[labels == label_with_largest_area] = 1

In [ ]:
def crop_zeros(image, img_mask_, get_coords=True):
    points = np.argwhere(img_mask_.sum(axis=0))
    left_boundary = points.min()
    right_boundary = points.max()
    
    points = np.argwhere(img_mask_.sum(axis=1))
    top_boundary = points.min()
    bottom_boundary = points.max()

    if get_coords:
        return image[top_boundary:bottom_boundary, left_boundary: right_boundary], (top_boundary, bottom_boundary, left_boundary, right_boundary)
    else:
        return image[top_boundary:bottom_boundary, left_boundary: right_boundary]

In [ ]:
dbt_cropped_image, dbt_coords = crop_zeros(dbt_image_stack_sum, island_mask)

In [ ]:
dbt_image_stack_cropped = dbt_image_stack[:, dbt_coords[0]:dbt_coords[1], dbt_coords[2]:dbt_coords[3]]

In [ ]:
fig, axs = plt.subplots(6, 7, figsize=(20, 20), sharey=True, sharex=True, tight_layout=True)

for i, ax in enumerate(axs.flatten()):
    ax.imshow(np.rot90(dbt_image_stack_cropped[i], 2), 'gray')
    ax.set_axis_off()
plt.show()

In [ ]:
animate_3d_array(np.rot90(dbt_image_stack_cropped, 2), vmin=dbt_image_stack_cropped.min(), vmax=dbt_image_stack_cropped.max())

# 6. MRI

In [ ]:
mri_sample = pd.read_csv("/fsx/embed/summer-school-24/Theo_session/tables/mri_sample_df.csv")

In [ ]:
mri_sample.StudyDescription.value_counts(dropna=False)

In [ ]:
mri_sample.SeriesDescription.value_counts(dropna=False)

In [ ]:
mri_sample_axial = mri_sample[mri_sample.SeriesDescription=="AX VIBRANT DYNAMIC"]

In [ ]:
mri_sample_axial = mri_sample_axial.sort_values(["InstanceNumber"])

In [ ]:
loaded_mri_dicoms = []
for dcm_path in tqdm(mri_sample_axial["anon_dicom_path"]):
    # print(dcm_path)
    loaded_mri_dicoms.append(pydicom.dcmread(dcm_path))

In [ ]:
pixel_spacing = loaded_mri_dicoms[0].PixelSpacing
slice_thickness = loaded_mri_dicoms[0].SliceThickness

In [ ]:
ax_aspect = pixel_spacing[1]/pixel_spacing[0]
sag_aspect = slice_thickness/pixel_spacing[0]
cor_aspect = slice_thickness/pixel_spacing[1]

In [ ]:
images = np.zeros((len(loaded_mri_dicoms), loaded_mri_dicoms[0].Rows, loaded_mri_dicoms[0].Columns))

In [ ]:
for i, dcm_ in enumerate(loaded_mri_dicoms):
    images[i] = process_dicom_image(dcm_)

In [ ]:
images_coronal = images.transpose((1, 0, 2))
images_sagital = images.transpose((2, 0, 1))

In [ ]:
fig, axs = plt.subplots(10, 5, figsize=(20, 40), sharey=True, sharex=True, tight_layout=True)
fig.suptitle("Axial", fontsize=24)
starting_slice = 30
skip_slice = 0
for i, ax in enumerate(axs.flatten()):
    ax.imshow(np.rot90(images[starting_slice+i+i*skip_slice], 2), 'gray', vmin=np.iinfo(np.int16).min, vmax=np.iinfo(np.int16).max)
    ax.set_aspect(ax_aspect)
    ax.set_axis_off()
plt.show()

In [ ]:
animate_3d_array(np.rot90(images, 2))

In [ ]:
fig, axs = plt.subplots(10, 5, figsize=(20, 40), sharey=True, sharex=True, tight_layout=True)
starting_slice = 90
skip_slice = 2
for i, ax in enumerate(axs.flatten()):
    ax.imshow(np.rot90(images_coronal[starting_slice+i+i*skip_slice], 2), 'gray', vmin=np.iinfo(np.int16).min, vmax=np.iinfo(np.int16).max)
    ax.set_aspect(cor_aspect)
    ax.set_axis_off()
plt.show()

In [ ]:
images_coronal_fliplr_flip_ud = np.flipud(np.fliplr(images_coronal[:240]))

In [ ]:
animate_3d_array(images_coronal_fliplr_flip_ud, aspect_ratio=cor_aspect)

In [ ]:
fig, axs = plt.subplots(10, 5, figsize=(20, 40), sharey=True, sharex=True, tight_layout=True)
starting_slice = 100
skip_slice = 5
for i, ax in enumerate(axs.flatten()):
    ax.imshow(np.rot90(images_sagital[starting_slice+i+i*skip_slice], 2), 'gray', vmin=np.iinfo(np.int16).min, vmax=np.iinfo(np.int16).max)
    ax.set_aspect(sag_aspect)
    ax.set_axis_off()
plt.show()

In [ ]:
animate_3d_array(np.rot90(images_sagital, 2), aspect_ratio=sag_aspect)

### ☢️ ***Exercise 4***

Load and visualize a true SAGITAL series.

What main differences do you see compared with the resliced Sagital view above?

In [ ]:
# Code your solution here